In [1]:
import pandas as pd

data = pd.read_csv(r'./esp_municipios_20200518.csv')
df = pd.DataFrame(data)

In [5]:
df

,CODIGO_CA,COMUNIDAD_AUTONOMA,Codigo Provincia,PROVINCIA,NUMERO_INSCRIPCION,Codigo Municipio,DENOMINACION,FECHA_INSCRIPCION,SUPERFICIE,HABITANTES,DENSIDAD,CAPITALIDAD,ANOTACION
0,1,Andalucía,13385,Almería,1040010,5517,Abla,16/02/87,"45,24",1249,"27,61",Abla,
1,1,Andalucía,13385,Almería,1040025,5518,Abrucena,22/04/87,"83,68",1202,"14,36",Abrucena,
2,1,Andalucía,13385,Almería,1040031,5519,Adra,13/11/86,"89,69",25148,"280,39",Adra,
3,1,Andalucía,13385,Almería,1040046,5520,Albanchez,22/04/87,"34,91",741,"21,23",Albanchez,\n \n
4,1,Andalucía,13385,Almería,1040059,5521,Alboloduy,19/01/87,"69,73",616,"8,83",Alboloduy,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8127,17,"Rioja, La",13434,"Rioja, La",1261807,9443,Zarratón,25/11/86,"18,66",264,"14,15",Zarratón,
8128,17,"Rioja, La",13434,"Rioja, La",1261814,9444,Zarzosa,20/10/86,"18,29",14,",77",Zarzosa,
8129,17,"Rioja, La",13434,"Rioja, La",1261835,9445,Zorraquín,23/02/87,"6,44",93,"14,44",Zorraquín,
8130,18,Ciudad de Ceuta,13435,Ceuta,1510013,13346,Ceuta,13/10/86,"19,87",84777,"4266,58",Ceuta,


In [25]:
cp_data = pd.read_csv(r'./cp.csv')
cd_df = pd.DataFrame(cp_data)
cd_df


,Unnamed: 0,provincia,poblacion,Código Postal
0,NaN,Araba/Álava,Alegría-Dulantzi,240
1,NaN,Ávila,Candeleda,548
2,NaN,Araba/Álava,Vitoria-Gasteiz,1001
3,NaN,Araba/Álava,Vitoria-Gasteiz,1002
4,NaN,Araba/Álava,Vitoria-Gasteiz,1003
...,...,...,...,...
14660,NaN,Melilla,Melilla,52004
14661,NaN,Melilla,Melilla,52005
14662,NaN,Melilla,Melilla,52006
14663,NaN,Tarragona,Calafell,73820


In [84]:
cp = cd_df[cd_df['poblacion'].str.contains('Huelva')]
#cp = cp[cp['Código Postal'].gt(10000)]
cp = cp['Código Postal'].tolist()
cp

[21000, 21001, 21002, 21003, 21004, 21005, 21006, 21007]

In [88]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='localhost', database='Comedores', user='root', password='secr3t!')

    if connection.is_connected():
        print('connected')
        cursor = connection.cursor()
        cursor.execute('INSERT INTO countries(name) SELECT "españa" FROM dual WHERE NOT EXISTS (SELECT name FROM countries WHERE name = %s)', ('españa', ))
        connection.commit()

        for row in df.itertuples():
            cursor.execute('INSERT INTO states(name, country_id) SELECT * FROM (SELECT %s AS name, 1 AS country_id) AS new_value WHERE NOT EXISTS (SELECT name FROM states WHERE name = %s) LIMIT 1', (row.COMUNIDAD_AUTONOMA, row.COMUNIDAD_AUTONOMA))
            connection.commit()

            cursor.execute('SELECT id, name FROM states WHERE name = %s', (row.COMUNIDAD_AUTONOMA, ))
            stateId = cursor.fetchone()[0]

            cursor.execute('INSERT INTO provinces(name, state_id) SELECT * FROM (SELECT %s AS name, %s AS state_id) AS new_value WHERE NOT EXISTS (SELECT name FROM provinces WHERE name = %s) LIMIT 1', (row.PROVINCIA, stateId, row.PROVINCIA))
            connection.commit()

            cursor.execute('SELECT id, name from provinces WHERE name = %s', (row.PROVINCIA, ))
            provinceId = cursor.fetchone()[0]

            cp = cd_df[cd_df['poblacion'].str.contains(row.DENOMINACION)]
            cp = cp['Código Postal'].tolist()
            cp = ','.join(map(lambda x: str(x), cp))

            cursor.execute('INSERT INTO cities(name, postal_code, province_id) SELECT * FROM (SELECT %s as name, %s AS postal_code, %s AS province_id) AS new_value WHERE NOT EXISTS (SELECT name FROM cities WHERE name = %s) LIMIT 1', (row.DENOMINACION, cp, provinceId, row.DENOMINACION))
            connection.commit()

        

except Error as e:
    print(e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print('disconnected')

connected


/tmp/ipykernel_25495/3440180756.py:26: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  cp = cd_df[cd_df['poblacion'].str.contains(row.DENOMINACION)]


disconnected
